<a href="https://colab.research.google.com/github/ryanleeallred/DS-Unit-1-Sprint-2-Statistics/blob/master/module1/LS_DS_121_Statistics_Probability_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

## *Data Science Unit 1 Sprint 2 Assignment 1*

# Apply the t-test to real data

Your assignment is to determine which issues have "statistically significant" differences between political parties in this [1980s congressional voting data](https://archive.ics.uci.edu/ml/datasets/Congressional+Voting+Records). The data consists of 435 instances (one for each congressperson), a class (democrat or republican), and 16 binary attributes (yes or no for voting for or against certain issues). Be aware - there are missing values!

Your goals:

1. Load and clean the data (or determine the best method to drop observations when running tests)
2. Using hypothesis testing, find an issue that democrats support more than republicans with p < 0.01
3. Using hypothesis testing, find an issue that republicans support more than democrats with p < 0.01
4. Using hypothesis testing, find an issue where the difference between republicans and democrats has p > 0.1 (i.e. there may not be much of a difference)

Note that this data will involve *2 sample* t-tests, because you're comparing averages across two groups (republicans and democrats) rather than a single group against a null hypothesis.

In [80]:
!ls
!head -n 100 house-votes-84.names

house-votes-84.data
house-votes-84.names
LS_DS_121_Statistics_Probability_and_Inference.ipynb
LS_DS_121_Statistics_Probability_Assignment.ipynb
notes.txt
1. Title: 1984 United States Congressional Voting Records Database

2. Source Information:
    (a) Source:  Congressional Quarterly Almanac, 98th Congress, 
                 2nd session 1984, Volume XL: Congressional Quarterly Inc. 
                 Washington, D.C., 1985.
    (b) Donor: Jeff Schlimmer (Jeffrey.Schlimmer@a.gp.cs.cmu.edu)
    (c) Date: 27 April 1987 

3. Past Usage
   - Publications
     1. Schlimmer, J. C. (1987).  Concept acquisition through 
        representational adjustment.  Doctoral dissertation, Department of 
        Information and Computer Science, University of California, Irvine, CA.
        -- Results: about 90%-95% accuracy appears to be STAGGER's asymptote
     - Predicted attribute: party affiliation (2 classes)

4. Relevant Information:
      This data set includes votes for each of the U.S. House of

In [3]:
x = "   6. el-salvador-aid: 2 (y,n)"
#pat = re.compile('\d+.\s([A-Za-z-]+):)
pat = re.compile(r'\s+\d+.\s([A-Za-z-\s]+):\s\d\s')

print(pat.match(x).group(1))


el-salvador-aid


In [34]:
### YOUR CODE STARTS HERE
from scipy import stats
import pandas as pd      #probably can do this all without dataframes but im feeling lazy
import numpy as np
import re
#LOAD DATA , CHECK FORMAT.
#using https://archive.ics.uci.edu/ml/datasets/Congressional+Voting+Records
#per the data description the "?" indicates that the member was present and either abstained or used 
#procedure to avoid voting on the issue therefore we are going to class this result with nays as in 
# 'not willing to support'
# An alternative woud be to simply calculate the ratio of yeas to nays, but i dont think this reflects the 
# reality of how voting works in practice. an absolute number of yeas is required to pass a bill
start = re.compile("7.\sAttribute")
pat = re.compile(r'\s+\d+.\s([A-Za-z-\s]+):\s\d\s')  #extract column names
columns=[]
with open('house-votes-84.names', 'r') as f:
    line = f.readline()
    cnt = 1
    while line:
        if start.match(line):
            line = f.readline()  #read next line and process. until done and break
            while pat.match(line):
            #    print(pat.match(line).group(1))  #print the match -> add to lis    
                line = f.readline()  #read next line and process.
                
        else:
            line = f.readline() #not started, read nextline
            cnt += 1
##that was fun
with open('house-votes-84.names', 'r') as f:
    txt = f.read()
    columns = pat.findall(txt)                   #this is what we want



In [213]:
df= pd.read_csv('house-votes-84.data', names=columns)
df=df.reset_index()      
df.rename({'index': 'voteinstance'}, axis =1, inplace=True)
df = df.melt(id_vars=['voteinstance','Class Name'], var_name='issue',value_name='vote')

filter = df['issue'] == 'handicapped-infants'
df.vote = df['vote'].apply(lambda v : 1 if (v == 'y') else 0)
df[filter].groupby(by='Class Name',axis=0).sum()
votecounts =  df.groupby(by=['issue','Class Name'],axis=0).sum()  #rename vote to support
votecounts.drop(labels='voteinstance', axis=1,inplace=True)

tot_voters = df[filter]["Class Name"].count()  # Total number voters per issue
voters_by_party = df[filter]["Class Name"].value_counts()  #number voters per party per issue
print(voters_by_party['democrat'])
#votecounts['party_percent'] = 
votecounts.dtypes
#.apply(lambda p,v : v / voters_by_party[p])

267


vote    int64
dtype: object

## Stretch Goals:

1. Refactor your code into functions so it's easy to rerun with arbitrary variables
2. Work on Performing a T-test without using Scipy in order to get "under the hood" and learn more thoroughly about this topic.
### Start with a 1-sample t-test
 - Establish the conditions for your test 
 - [Calculate the T Statistic](https://blog.minitab.com/hs-fs/hubfs/Imported_Blog_Media/701f9c0efa98a38fb397f3c3ec459b66.png?width=247&height=172&name=701f9c0efa98a38fb397f3c3ec459b66.png) (You'll need to omit NaN values from your sample).
 - Translate that t-statistic into a P-value. You can use a [table](https://www.google.com/search?q=t+statistic+table) or the [University of Iowa Applet](https://homepage.divms.uiowa.edu/~mbognar/applets/t.html)

 ### Then try a 2-sample t-test
 - Establish the conditions for your test 
 - [Calculate the T Statistic](https://lh3.googleusercontent.com/proxy/rJJ5ZOL9ZDvKOOeBihXoZDgfk7uv1YsRzSQ1Tc10RX-r2HrRpRLVqlE9CWX23csYQXcTniFwlBg3H-qR8MKJPBGnjwndqlhDX3JxoDE5Yg) (You'll need to omit NaN values from your sample).
 - Translate that t-statistic into a P-value. You can use a [table](https://www.google.com/search?q=t+statistic+table) or the [University of Iowa Applet](https://homepage.divms.uiowa.edu/~mbognar/applets/t.html)

 ### Then check your Answers using Scipy!

In [ ]:
im

In [ ]:
  # RUN TH  TTEST against 2 classes